# WHY FAKE NEWS IS A PROBLEM?
**Fake news refers to misinformation, disinformation or mal-information which is spread through word of mouth and traditional media and more recently through digital forms of communication such as edited videos, memes, unverified advertisements and social media propagated rumours.Fake news spread through social media has become a serious problem, with the potential of it resulting in mob violence, suicides etc as a result of misinformation circulated on social media.**
![image.png](attachment:image.png)

# BRIEF DESCRIPTION OF DATASET
**This dataset consists of about 40000 articles consisting of fake as well as real news. Our aim is train our model so that it can correctly predict whether a given piece of news is real or fake.The fake and real news data is given in two separate datasets with each dataset consisting around 20000 articles each.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls
%cd /content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset

drive  sample_data
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


In [4]:
%cd fake-and-real-news-dataset/

[Errno 2] No such file or directory: 'fake-and-real-news-dataset/'
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


# LOADING THE NECESSARY LIBRARIES

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# IMPORTING THE DATASET

# DATA VISUALIZATION AND PREPROCESSING

**Import data in tf dataset**

In [6]:
true_dataset = tf.data.experimental.CsvDataset(filenames = ['True.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)
false_dataset = tf.data.experimental.CsvDataset(filenames = ['Fake.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)

In [7]:
for idx, row in enumerate(true_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b'As U.S. budget fight looms, Republicans flip their fiscal script'>, <tf.Tensor: shape=(), dtype=string, numpy=b'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a \xe2\x80\x9cfiscal conservative\xe2\x80\x9d on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS\xe2\x80\x99 \xe2\x80\x9cFace the Nation,\xe2\x80\x9d drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. Pr

**Join text and title**

In [8]:
def preprocess_join_sentence(s1, s2):
    ret = tf.strings.reduce_join([s1, s2],separator=" ", axis=-1)
    return ret

true_dataset = true_dataset.map(preprocess_join_sentence)
false_dataset = false_dataset.map(preprocess_join_sentence)

In [9]:
for idx, row in enumerate(true_dataset):
    print (row)
    if idx > 3:
        break

tf.Tensor(b'As U.S. budget fight looms, Republicans flip their fiscal script WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a \xe2\x80\x9cfiscal conservative\xe2\x80\x9d on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS\xe2\x80\x99 \xe2\x80\x9cFace the Nation,\xe2\x80\x9d drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military 

### Add labels to dataset

In [10]:
cat_true = tf.data.Dataset.from_tensor_slices(np.ones(len(list(true_dataset)), dtype=int))
cat_false = tf.data.Dataset.from_tensor_slices(np.zeros(len(list(false_dataset)), dtype=int))

In [11]:
true_dataset = tf.data.Dataset.zip((true_dataset, cat_true))
false_dataset = tf.data.Dataset.zip((false_dataset, cat_false))

In [21]:
full_dataset = false_dataset.concatenate(true_dataset)

In [13]:
for idx, row in enumerate(full_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b' Donald Trump Sends Out Embarrassing New Year\xe2\x80\x99s Eve Message; This is Disturbing Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What k

**Formate sentences**

In [22]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def rm_stopwords(text):
  stop = set(stopwords.words('english'))
  nltk.download('punkt')

  stop = list(stop)
  ret = tf.strings.strip(text)

  for i in stop:
    ret = tf.strings.regex_replace(ret, '\\b'+i+'\\b', '')
  # Remove adding spaces

  ret = tf.strings.regex_replace(ret, ' +', ' ')
  return ret

def rm_poncutation(text):
  punctuation = list(string.punctuation)
  punctuation.remove('(')
  punctuation.remove(')')
  punctuation.remove(',')
  punctuation.remove('[')
  punctuation.remove(']')
  print(punctuation)

  ret = tf.strings.strip(text)
  for i in punctuation:
    ret = tf.strings.regex_replace(ret, '\\b'+i+'\\b', '')
  # Remove adding spaces
  # ret = tf.strings.regex_replace(ret, ' +', ' ',)
  return ret

def preprocess_sentence(sentence, label):
  ret = tf.strings.lower(sentence)
  # ret = ret.decode(utf-8)
  # ret = BeautifulSoup(ret, "html.parser").get_text()
  ret = tf.strings.strip(ret)
  ret = tf.strings.regex_replace(ret, r'[^\x00-\x7f]','')
  ret = tf.strings.regex_replace(ret, '<[^<]+?>', '') # Removing HTML chars
  ret = tf.strings.regex_replace(ret, '\[[^]]*\]', '') # Removing the square brackets
  ret = tf.strings.regex_replace(ret, 'http\S+', '') # Removing URL's
  # ret =  tf.strings.regex_replace(ret, '[^\w\s]','') # Remove punctuation
  ret = rm_stopwords(ret)
  # ret = rm_poncutation(ret)
  return ret, label

full_dataset = full_dataset.map(preprocess_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
for idx, row in enumerate(full_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b'donald trump sends embarrassing new years eve message; disturbing donald trump wish americans happy new year leave . instead, give shout enemies, haters dishonest fake news media. former reality show star one job . country rapidly grows stronger smarter, want wish friends, supporters, enemies, haters, even dishonest fake news media, happy healthy new year, president angry pants tweeted. 2018 great year america! country rapidly grows stronger smarter, want wish friends, supporters, enemies, haters, even dishonest fake news media, happy healthy new year. 2018 great year america! donald j. trump (@realdonaldtrump) december 31, 2017trump tweet went welll expect. kind president sends new year greeting like despicable, petty, infantile gibberish? trump! lack decency even allow rise gutter long enough wish american citizens happy new year! bishop talbert swan (@talbertswan) december 31, 2017no one likes calvin (@calvinstowell) december 31, 2017your

In [24]:
!pip install -q tensorflow-text


     |████████████████████████████████| 3.4MB 4.2MB/s 


### Glove

**Count max occurences in dataset**

In [25]:
from collections import Counter

def freq_words(inputs, max_size=15000): # input dataset, returns list
  count = Counter()

  numpy_sentences = []
  for sentences, labels in full_dataset.take(-1):  # take all elements in dataset
      numpy_sentences.extend(tf.strings.split(sentences).numpy())
      numpy_labels = labels.numpy()
  list_words = numpy_sentences

  for word in list_words:
    # print(word)
    count[word] += 1
  result = [word for word, num in count.most_common(max_size)]
  return result

max_freq_words = freq_words(full_dataset)

In [26]:
max_freq_words = [w.decode("utf-8") for w in max_freq_words]

In [27]:
max_freq_words[:10]

['trump',
 'said',
 ',',
 '.',
 'would',
 'u..',
 'president',
 'people',
 'one',
 'new']

**Tests**

In [28]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Build vocabulary and relevent vector
words = []
# Adding 2 empty vector of dimension 100 which will be used in padding and unknown token(word)
# According to previous knowledge i have idea that it contains 400,000 words
# Added extra 2 rows because when we will use inside TextVectorization add blank i.e. '' and ['UNK'] inside vocabulary

counter = 0

vectors = np.zeros((10000, 100))

with open(f"../glove-twitter/glove.twitter.27B.100d.txt") as f:
    lines = f.readlines()

    for idx, line in enumerate(lines):
      if counter >= 10000-2:
        break

      split_line = line.split()
      
      if split_line[0] in max_freq_words:
        words.append(split_line[0])
        vectors[counter+2] = split_line[1:]
        counter+=1

print(f"Number of words are {len(words)}")
print(f"Shape of vector is {vectors.shape}")

Number of words are 9998
Shape of vector is (10000, 100)


In [29]:
vectors

array([[ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 1.8205e-01, -4.8483e-02,  2.3966e-01, ..., -3.3580e-01,
         1.8884e-01, -4.0786e-01],
       ...,
       [ 3.4225e-01,  1.0624e+00,  5.2395e-01, ...,  9.9312e-02,
         4.9109e-01,  1.2910e-01],
       [ 2.8270e-01,  3.5967e-01,  1.2944e+00, ...,  1.1320e+00,
        -4.6097e-01,  2.9973e-01],
       [ 3.2018e-01, -8.1367e-01,  6.8017e-01, ...,  1.1296e-03,
        -5.3972e-01, -2.0115e-01]])

In [30]:
words[:10]

['.', ':', 'rt', ',', '!', '"', '?', '(', ')', 'de']

In [31]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# TextVectorization hyper parameter
max_vocab = 10000
max_len = 300
vectorize_layer_glove = TextVectorization(max_tokens=max_vocab,
                                                    standardize="lower_and_strip_punctuation",
                                                    split="whitespace",
                                                    output_mode="int",
                                                    output_sequence_length=max_len,
                                                    vocabulary=words)


# Vocabulary set into layer
vectorize_layer_glove.set_vocabulary(words)
print(f"Top 10 words {vectorize_layer_glove.get_vocabulary()[:10]}")
print(f"Some words {vectorize_layer_glove.get_vocabulary()[-1010:-1000]} \n")

Top 10 words ['', '[UNK]', '.', ':', 'rt', ',', '!', '"', '?', '(']
Some words ['defect', 'emphasize', 'southeastern', 'conveniently', 'feinstein', 'imposed', 'attackers', 'abandoning', 'distinguished', 'angered'] 



In [32]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer_glove(text), label

train_data_tokenized = full_dataset.map(int_vectorize_text)

In [33]:
model_glove = tf.keras.models.Sequential()
model_glove.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string))
model_glove.add(vectorize_layer_glove)
model_glove.predict(["expect terror busts fbi steps use isis stings 21st century wire says real terrorist threat thing create one received wisdom today federal bureau investigationaccording latest public relations release fbi counter terrorism department agency plans significantly increased use stings order catch wouldbe terrorists words planning rampup policy using entrapment order organize recruit train even supply materials budding neophyte terrorists usthis come surprise anyone following incidents spite grand announcement reality fbi last 25 years stings stings stings fbi direct james comey spend lot time front congress year logic astounding insane reason type antiquated thinking still dominated us federal law enforcement institutions going wait person mobilize time line said michael b steinbach leads fbi national security branch added fbi could afford sit wait knowing individual actively plotting since 911 fbi agents relied practice using informants pose jihadi radicals small arms dealers order excite recruit fringe members society constructive pursuits bombmaking often bring scene mock terrorists crime agents leap behind shadows make highprofile arrest unwitting dupesuspectin 2016 recruitment required assemble theatrical statemanaged scenarios done online via friends social media fbi cyber agents look individuals appear supporting isis alqaeda successful online grooming transitions real world meeting luck blossoms contrived terror plot courtesy fbi confidential informant 1993 wtc bomber ramzi yousefperhaps bestever example fbi sting resulting real terrorist attack event mindless us mainstream media still wrongly attribute real terrorists 1993 world trade center bombingin allfbihatched plot agency used least one known informant man named emad salem former egyptian army officer salem later demonstrated secret recording made fbi originally asked supply conspirators harmless powder instead actual explosive construct bomb reasons still officially unknown fbi chose allow real bomb deployed wtc day supposed sting author tom secker global research explains simplest terms relatively ragtag group inspired cia asset blind sheikh trained triple agent ali mohamed managed pull bombing result bomb detonating underneath wtc putting thousands innocent americans riskstill ever single major us television newspaper outlet still moronically refers 1993 wtc event terrorist attack american soil team america fabricated terroraccording fbi records nearly 70 reported isislinked terrorist prosecutions convictions cookedup via use informants stingsentrapment undercover operations seen last resort used two every three prosecutions involving people suspected supporting islamic state sharp rise span two years according new york times analysis charges brought nearly 90 americans believed linked group back may fbi arrested man james medina allegedly planning attack miamiarea aventura turnberry jewish center synagogue explosive device turns synagogue never actually real danger bomb fake sold fbi undercover agent man stopped man stopped fbi agents posing fellow terrorists set sting operation fbi chatup lines muslim mosque go know alqaeda 2015 one classic exhibit frameup 20 year old christopher lee cornell snared one bogus fake terror plot ever yet us mainstream media stop gushing event isisinspired attack attack actually took placeaccording david steele 20year marine corps intelligence officer formercia case officer us terror plots cooked us authorities terrorists false flag terrorists created security services united states every single terrorist incident false flag informant pushed fbi fact citizens taking restraining orders fbi informants trying incite terrorism become lunatic asylum similar wouldbe highprofile terror icons patsies include paris shooter amedi coulibaby see incredible informant case ottawa shooter zehafbibeau see patsy story boston bomber tamerlan tsarnaev see informant story underwear bomber umar farouk abdulmutallab see patsy story buford rogers read informant story jerad miller read informant story naji mansour read entrapment story quazi mohammad nafis read patsy story mohamed osman mohamud read entrapment story even infamous okc bomber timothy mcveigh read sheepdipped patsy story herein reality covert culture washington dc promote genuine terrorism home internationally much single nation state outside saudi arabia israel major people realize thisread zany fbi plots 21st century wire daily shooter filessupport work become 21wiretv member"])

array([[ 948, 2673,    1, 3925, 2365,  306, 6432,    1,    1, 3474, 4062,
         343,  117, 4277, 3703,  165, 1614,   32, 2074, 2519,   52, 2844,
        6130,    1, 1327, 1038, 4738, 1242, 3925, 3983, 5577, 3783, 2939,
        1002, 7296, 4833,  306,    1,  762,  729,    1, 5122,  447, 1861,
           1, 3184,  745,    1,  762, 5926, 6216,  542,   88, 3433, 6322,
           1,    1, 5122,    1,   89, 1388,  264,  395, 7484, 6531, 1200,
        3149, 1273, 3925,  106,    1,  268,    1,    1,    1, 3925, 1704,
         741, 7563,  803,  279,   47,  751, 2891,  143, 3048, 8440, 1963,
         407,  654,    1,  357,   82, 6947,  108, 2844, 1271, 6750, 7790,
          78,  159,  169, 9982,   47,  510,  162,  826,   26,    1, 2400,
        3925, 1043, 1758, 4988, 1447, 3925,  113, 2859,  703,  159, 1044,
        3189, 7626, 6687,  324,    1, 3925, 5249, 9779,  994,  745,    1,
        2645, 9988, 9209,  720, 1442, 6477,  762,    1, 6216, 4920, 1898,
        2071, 7848,    1,    1, 1264, 

In [ ]:
for idx, row in enumerate(train_data_tokenized):
    print (row)
    if idx > 0:
        break

In [35]:
len(vectors[0])

100

### Prepare data to train

In [36]:
# Model params
batch_size = 256
epochs = 10
embed_size = 100
maxlen = 300
max_features = 10000

**Model**

In [37]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True))
model.add(LSTM(units=64))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)


In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 300, 128)          117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________


### TODO : add validation dataset and shuffle data

In [41]:
full_dataset = train_data_tokenized.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

In [40]:
len(list(full_dataset))

44898

In [46]:
DATASET_SIZE = len(list(full_dataset))

train_size = int(0.8 * DATASET_SIZE)
test_size = int(0.2 * DATASET_SIZE)

full_dataset = full_dataset.shuffle(buffer_size=DATASET_SIZE, seed=42)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)

In [ ]:
for idx, row in enumerate(train_dataset):
    print (row)
    if idx > 3:
        break

In [44]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17524386010747095968
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11153436864
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17655670020707594348
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [ ]:
# Train for 2 epochs
model.fit(train_dataset, validation_data=test_dataset, epochs=2, steps_per_epoch=25000)